In [ ]:
%cd ./FinanceRAG

In [ ]:
import pandas as pd
from financerag.common import HFDataLoader

In [3]:
from tqdm import tqdm
from openai import OpenAI
import os
import json
import pandas as pd
import re
import time
from datasets import Dataset

In [ ]:
dataset_path = "../dataset"
subset = "MultiHiertt"

corpus, _= HFDataLoader(
    data_folder=dataset_path,
    subset=subset,
    keep_in_memory=False,
).load()

uni_chr_re = re.compile(r'\\u([a-fA-F0-9]{4})')
final_corpus = {doc["id"]: {"title": doc["title"], "text": doc["text"]} for doc in corpus}
tabular_corpus = {}
corpus_ids = list(final_corpus.keys())


for i in tqdm(range(len(corpus_ids))):
  corpus_id = corpus_ids[i]
  tabular_corpus[corpus_id] = {}
  tabular_corpus[corpus_id]["title"] = final_corpus[corpus_id]["title"]
  tabular_corpus[corpus_id]["text"] = uni_chr_re.sub(lambda m: chr(int(m.group(1), 16)), final_corpus[corpus_id]["text"])
tabular_corpus_dataset = [{"_id": doc_id, "title": doc["title"], "text": doc["text"]} for doc_id, doc in tabular_corpus.items()]
tabular_corpus_hf = Dataset.from_list(tabular_corpus_dataset)
save_path = f'../dataset/MultiHiertt/corpus_1207_unicode.jsonl'
with open(save_path, 'w', encoding='utf-8') as f:
  json.dump(tabular_corpus, f, ensure_ascii= True)

In [ ]:
save_path = f'../dataset/MultiHiertt/corpus_1207_unicode.jsonl'
with open(save_path, 'r', encoding='utf-8') as f:
    dataset_multi = json.load(f)
tabular_corpus_dataset = [{"_id": doc_id, "title": doc["title"], "text": doc["text"]} for doc_id, doc in dataset_multi.items()]
tabular_corpus_hf = Dataset.from_list(tabular_corpus_dataset)
tabular_corpus_hf.to_json(save_path)

In [ ]:
def generate_output(system_prompt, user_prompt):
    client = OpenAI(api_key="")

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0
    )
    text = response.choices[0].message.content

    return text



global call_count
call_count = 0

# MultiHiertt
def replace_multi_table(text):
    call_count = 0
    instruction = """You are given a hierarchical table. Change each cell into a sentence, along with its hierarchical row and column headers.
                    Don't dismiss hierarchical column headers and row headers. Cells are divided by "|".
                    Do not indexing for the results. """

    new_text = generate_output(instruction, text)

    call_count += 1
    if call_count % 1000 == 0:
        time.sleep(60)  # Pause for 1 minute
    return new_text

In [ ]:
dataset_path = "Linq-AI-Research/FinanceRAG"
subset = "MultiHiertt"

corpus, _= HFDataLoader(
    hf_repo=dataset_path,
    subset=subset,
    keep_in_memory=False,
).load()

final_corpus = {doc["id"]: {"title": doc["title"], "text": doc["text"]} for doc in corpus}
tabular_corpus = {}
corpus_ids = list(final_corpus.keys())


for i in tqdm(range(len(corpus_ids))):
  corpus_id = corpus_ids[i]
  tabular_corpus[corpus_id] = {}
  tabular_corpus[corpus_id]["title"] = final_corpus[corpus_id]["title"]
  tabular_corpus[corpus_id]["text"] = replace_multi_table(final_corpus[corpus_id]["text"])
tabular_corpus_dataset = [{"_id": doc_id, "title": doc["title"], "text": doc["text"]} for doc_id, doc in tabular_corpus.items()]
tabular_corpus_hf = Dataset.from_list(tabular_corpus_dataset)
save_path = f'../dataset/MultiHiertt/corpus_1207.jsonl'
tabular_corpus_hf.to_json(save_path)